# Fly-tipping Report

This task requires using the UK government's [fly-tipping dataset](https://www.data.gov.uk/dataset/1388104c-3599-4cd2-abb5-ca8ddeeb4c9c/fly-tipping-in-england) to investigate the following questions:

- Which kind of incidents happen most in your region?
- How many people have been sent to prison in your region?
- In which region is flytipping growing the fastest?
- Which region has the biggest fluctuations in clean costs?
- Which region is losing the most money from flytipping?

In [10]:
import urllib.request
import os

FLYTIPPING_ACTIONS_URL = "https://s3.eu-west-1.amazonaws.com/data.defra.gov.uk/statistics_2021/LA_flytipping_actions_2012-13_to_2020-21.csv"
FLYTIPPING_INCIDENTS_URL = "https://s3.eu-west-1.amazonaws.com/data.defra.gov.uk/statistics_2021/LA_flytipping_incidents_2012-13_to_2020-21_updated.csv"

def is_flytipping_data_downloaded(incidents_file_name="flytipping-incidents.csv", actions_file_name="flytipping-actions.csv"):
    """Check if the required datasets are downloaded in the local directory, 
    if not downloads them.
    """
    if not os.path.isfile(incidents_file_name) and not os.path.isfile(actions_file_name):
        urllib.request.urlretrieve(FLYTIPPING_INCIDENTS_URL, incidents_file_name)
        urllib.request.urlretrieve(FLYTIPPING_ACTIONS_URL, actions_file_name)
    else:
        return "Files already in local directory"
    

is_flytipping_data_downloaded()

'Files already in local directory'

In [19]:
import pandas as pd

df_incidents = pd.read_csv("./flytipping-incidents.csv", encoding="ISO-8859-1", header=1)

df_actions = pd.read_csv("./flytipping-actions.csv", encoding="ISO-8859-1", header=1)